In [80]:
from psycopg2 import DatabaseError
from psycopg2.extras import DictCursor

from idb.postgres_backend.db import PostgresDB, RecordSet

db = PostgresDB()

In [81]:
from __future__ import print_function
import feedparser
from pyquery import PyQuery as pq
import argparse

In [82]:
feed_to_parse = 'https://scan-bugs.org:443/portal/webservices/dwc/rss.xml'

In [83]:
if not feed_to_parse.startswith('http'):
    print ()
    print ("* non-HTTP feed supplied, assuming local file. *")

feed = feedparser.parse(feed_to_parse)


def get_title(entry):
    if "title" in entry:
        return(entry["title"])
    else:
        return("NO TITLE FOUND")

def get_pubDate(entry):
    if "published" in entry:
        return (entry["published"])
    else:
        return ("NO PUBLISHED DATE FOUND")


def get_id(entry):
    if "id" in entry:
        return (entry["id"])
    else:
        return ("NO ID FOUND")

def get_dataset_link(entry):
    if "ipt_dwca" in entry:
        return (entry["ipt_dwca"])
    elif "link" in entry:
        return (entry["link"])
    else:
        return ("NO DATASET LINK FOUND")

def get_eml_link(entry):
    if "ipt_eml" in entry:
        return (entry["ipt_eml"])
    elif "emllink" in entry:
        return (entry["emllink"])
    else:
        return ("NO EML LINK FOUND")

In [84]:
print (feed_to_parse)
if "title" in feed['feed']:
    print (feed['feed']['title'])
else:
    print ("Feed has no TITLE.")

https://scan-bugs.org:443/portal/webservices/dwc/rss.xml
SCAN Darwin Core Archive rss feed


In [85]:
for entry in feed.entries:
    entry_title = ""
    entry_pubDate = ""
    entry_id = ""
    entry_dataset_link = ""
    entry_eml_link = ""
    # feedparser converts many common fields into normalized names. Examples:
    #   guid --> id
    #   pubDate --> published
    #
    # Fields that contain colons such as ipt:dwca and ipt:eml get underscored to ipt_dwca and ipt_eml
    #
    # The actual IPT guid field is not visible as a normalized field since another id field is used.
    # However, the id is embedded in the middle of the id url so human can pluck it out if needed.


    print ("title:        ", get_title(entry).encode('utf-8').strip())
    print ("published:    ", get_pubDate(entry).encode('utf-8').strip())
    print ("id:           ", get_id(entry).encode('utf-8').strip())
    print ("dataset link: ", get_dataset_link(entry).encode('utf-8').strip())
    print ("eml link:     ", get_eml_link(entry).encode('utf-8').strip())

title:         ABS-ARTHARCH DwC-Archive
published:     Wed, 17 May 2023 05:05:39
id:            https://scan-bugs.org:443/portal/webservices/dwc/f52df103-0126-47fd-8836-934641444fcd
dataset link:  https://scan-bugs.org:443/portal/content/dwca/ABS-ARTHARCH_DwC-A.zip
eml link:      https://scan-bugs.org:443/portal/content/dwca/ABS-ARTHARCH_DwC-A.eml
title:         ANSP-ENT DwC-Archive
published:     Wed, 17 May 2023 05:05:39
id:            https://scan-bugs.org:443/portal/webservices/dwc/af7140c3-4aa2-41ac-b3e9-4c7415b3ce90
dataset link:  https://scan-bugs.org:443/portal/content/dwca/ANSP-ENT_DwC-A.zip
eml link:      https://scan-bugs.org:443/portal/content/dwca/ANSP-ENT_DwC-A.eml
title:         ANSP-ORTH DwC-Archive
published:     Thu, 06 Jul 2023 08:12:25
id:            https://scan-bugs.org:443/portal/webservices/dwc/e4c5903a-a7ef-4adc-96b0-71aa79c16f5d
dataset link:  https://scan-bugs.org:443/portal/content/dwca/ANSP-ORTH_DwC-A.zip
eml link:      https://scan-bugs.org:443/portal/cont

In [89]:
rs_matches = {}
entry_ids = []

for entry in feed.entries:
    entry_id = get_id(entry).encode('utf-8').strip()
    entry_id = entry_id.split('dwc/')[1]    
    if entry_id not in entry_ids:
        entry_ids.append(entry_id)
        print(entry_id)
        rs_sql = "select id, uuid::text as uuid, publisher_uuid, name, recordids, eml_link, file_link, ingest, first_seen, last_seen, pub_date, file_harvest_date, file_harvest_etag, eml_harvest_date, eml_harvest_etag, ingest_is_paused, ingest_paused_date from recordsets where publisher_uuid = '076c0ff6-65e9-48a5-8e4b-2447936f9a1c' and array_to_string(recordids, ' ') like '%{0}%'".format(entry_id)
        print(rs_sql)
        rs = db.fetchall(rs_sql)
        print(rs)
        rs_matches[str(entry_id)] = rs
        


f52df103-0126-47fd-8836-934641444fcd
select id, uuid::text as uuid, publisher_uuid, name, recordids, eml_link, file_link, ingest, first_seen, last_seen, pub_date, file_harvest_date, file_harvest_etag, eml_harvest_date, eml_harvest_etag, ingest_is_paused, ingest_paused_date from recordsets where publisher_uuid = '076c0ff6-65e9-48a5-8e4b-2447936f9a1c' and array_to_string(recordids, ' ') like '%f52df103-0126-47fd-8836-934641444fcd%'
[]
af7140c3-4aa2-41ac-b3e9-4c7415b3ce90
select id, uuid::text as uuid, publisher_uuid, name, recordids, eml_link, file_link, ingest, first_seen, last_seen, pub_date, file_harvest_date, file_harvest_etag, eml_harvest_date, eml_harvest_etag, ingest_is_paused, ingest_paused_date from recordsets where publisher_uuid = '076c0ff6-65e9-48a5-8e4b-2447936f9a1c' and array_to_string(recordids, ' ') like '%af7140c3-4aa2-41ac-b3e9-4c7415b3ce90%'
[]
e4c5903a-a7ef-4adc-96b0-71aa79c16f5d
select id, uuid::text as uuid, publisher_uuid, name, recordids, eml_link, file_link, inge

IndexError: list index out of range

In [88]:
rs_matches

{'0006a383-40ab-41dc-887f-617c97c490f6': [],
 '002c13d2-80a9-47db-a28b-4d2b94b7ec96': [],
 '0197093b-f523-468f-8e90-6f94faa24395': [],
 '028347c0-a33b-4a6f-9ecd-535358c5fbac': [],
 '05db5b76-3037-46db-ba55-79607d9fc050': [],
 '0d8689b5-5476-478d-a334-e44f205036c9': [],
 '11c99ac1-da27-4232-88f0-6e0da3c5f4c1': [],
 '120bf21c-d066-48ee-b4c0-880596ea22e4': [],
 '15cdb244-b82d-4798-af5e-ef7bc138f233': [],
 '190d5a14-8148-4a50-bfd8-5323fae78671': [],
 '1f4f2b80-fcdf-4866-8bce-f796862170ef': [],
 '20161031-4d22-4001-976d-dcf11874a52e': [],
 '2ab896ab-8b52-404a-b1d4-fad3fcbd37b4': [],
 '3a89cc92-38e7-4f79-9177-3891869ee2a0': [],
 '40f010d6-4e3c-464c-bac4-50e8f59c2d85': [],
 '539bc75f-9d43-4460-848d-93375a7eb94a': [],
 '5698d7d7-64cd-4ccf-9cc1-5e07a83e27d6': [],
 '57d5a1e6-f5e6-44aa-b1a4-4141f56297c1': [],
 '5eaa605c-4043-4522-9668-9f6ca08cc6eb': [],
 '5ebde68b-fc0f-466a-86c4-622b66981663': [],
 '601398a5-bdae-4361-a754-78e2d22f287b': [],
 '602df614-d246-476d-9e62-51ad74b36175': [],
 '610419d0

In [9]:
entry_ids

['992ca726-fd6e-4398-a8c6-07dcf1f71083',
 'b235bd87-fd7a-4c52-a5f7-79008c393131',
 'e3570181-82db-489c-b146-d8cb4b33e472',
 '41b2f1af-9b4e-450d-8f6c-e5df18e49711',
 '335643f7-6835-41fd-9dca-c6b8e7918780',
 '741d3036-156f-49fa-b672-1d7aad6d2810',
 'c7263105-7706-4d44-9384-8d0ef527a33a',
 '1ddcac88-7c7d-44c3-8bc9-3435aceeb615',
 'b8c8c26f-e836-4227-88bb-fa6b2446edb3',
 '3ae42fd2-0827-4f35-9ec5-89d26914b5a5',
 '2bd20e20-fae7-46e6-a496-5a9d71e962a2',
 '16fca9c2-f622-4cb1-aef0-3635a7be5aeb',
 'cda97de1-b5cc-4963-b497-a1ee6312cf77',
 '37bed406-b435-4539-b319-be2fdfb75b32',
 '0ca2afb9-24f1-48da-b5aa-e3b1727236f5',
 'ed38f34c-6591-4043-8e67-1f4597e2b903',
 'a9059e97-59da-48f3-8f7f-aec3ddfac069',
 'bf3e5fc5-8a24-4a59-88e6-cb12cae48258',
 '25b650b5-ada6-47ae-b438-b33707f97724',
 '7571ba64-4f12-4af9-a9ab-d2eba06b3af6',
 'de06a5b7-ffe2-40c4-b376-bb9b779bbdc9',
 '85d00be2-3f99-49c2-9185-3942fca4c5dc',
 '847b0b2e-b6d5-4373-8c09-0ab3bc9b8376',
 '324e42d3-bb41-4cff-b067-7bd55983d5a7',
 'ff59ebc1-27ef-

In [28]:
current_rs = {}

for key in entry_ids:
    rs_sql = "select id, uuid, publisher_uuid, name, recordids, eml_link, file_link, ingest, first_seen, last_seen, pub_date, file_harvest_date, file_harvest_etag, eml_harvest_date, eml_harvest_etag, ingest_is_paused, ingest_paused_date from recordsets where array_to_string(recordids, ' ') like '%{0}%'".format(key)
    # Fetch all the rows as a list of dictionaries
   #print(rs_sql)
    rs = [dict(row) for row in db.fetchall(rs_sql)]
    if key not in current_rs:
        current_rs[key] = rs
    #print(key)


In [32]:
len(current_rs)

105

In [57]:
current_rs

{'0106ca7b-218e-47a2-8ccd-bddcc0000278': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ecdysis.org/content/dwca/UA-RLMPC_DwC-A.eml',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   'file_link': u'https://ecdysis.org/content/dwca/UA-RLMPC_DwC-A.zip',
   'first_seen': datetime.datetime(2022, 8, 9, 23, 42, 41, 178849),
   'id': 47584L,
   'ingest': False,
   'ingest_is_paused': False,
   'ingest_paused_date': None,
   'last_seen': datetime.datetime(2024, 5, 10, 16, 4, 16, 305874),
   'name': u'UA-RLMPC DwC-Archive',
   'pub_date': datetime.datetime(2024, 5, 1, 20, 2, 47),
   'publisher_uuid': '0e8eb850-aa63-4a4d-b0b7-2ec820ef808f',
   'recordids': [u'0106ca7b-218e-47a2-8ccd-bddcc0000278'],
   'uuid': None}],
 '02019e22-ae05-4a3b-ab03-9dd5c0732a7e': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ecdysis.org/content/dwca/NMDG-NMDG_DwC-A.eml',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   '

In [66]:
current_rs['992ca726-fd6e-4398-a8c6-07dcf1f71083']

[{'eml_harvest_date': None,
  'eml_harvest_etag': None,
  'eml_link': u'https://ecdysis.org/content/dwca/AHAB_DwC-A.eml',
  'file_harvest_date': None,
  'file_harvest_etag': None,
  'file_link': u'https://ecdysis.org/content/dwca/AHAB_DwC-A.zip',
  'first_seen': datetime.datetime(2023, 12, 1, 0, 39, 12, 799448),
  'id': 140337L,
  'ingest': False,
  'ingest_is_paused': False,
  'ingest_paused_date': None,
  'last_seen': datetime.datetime(2024, 5, 10, 16, 4, 16, 215759),
  'name': u'AHAB DwC-Archive',
  'pub_date': datetime.datetime(2024, 5, 1, 19, 34, 5),
  'publisher_uuid': '0e8eb850-aa63-4a4d-b0b7-2ec820ef808f',
  'recordids': [u'992ca726-fd6e-4398-a8c6-07dcf1f71083'],
  'uuid': None}]

In [56]:
#create lookup table

def lookupURL(uuid):
        if uuid == 'e73fedf0-90ee-4c6d-88dd-49399878fc54':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=133','archive': 'https://ecdysis.org/content/dwca/ANSP-ENT_DwC-A.zip'}
        if uuid == 'f4666aa7-93f0-41f7-83c2-497af7a06887':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=102','archive': 'https://ecdysis.org/content/dwca/BROW-BCIC_DwC-A.zip'}
        if uuid == '6acaebd6-e8d2-454a-8f50-e7a6734b8c79':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=55','archive': 'https://ecdysis.org/content/dwca/CSEB-CSEB_DwC-A.zip'}
        if uuid == '6075f4b7-4242-4886-be7d-391614fffe41':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=111','archive': 'https://ecdysis.org/content/dwca/CU-CUAC_DwC-A.zip'}
        if uuid == '0395f3c4-0277-43b6-a36d-e07720588790':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=105','archive': 'https://ecdysis.org/content/dwca/DBG-DBGA_DwC-A.zip'}
        if uuid == '5aac25d2-bcfb-4084-a700-584311ea539d':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=132','archive': 'https://ecdysis.org/content/dwca/DMNS-Arac_DwC-A.zip'}
        if uuid == '55b17f44-8c6b-4dc5-a31d-d9955b425790':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=149','archive': 'https://ecdysis.org/content/dwca/DMNS-Ento_DwC-A.zip'}
        if uuid == 'f8d0dae7-8927-4f5f-9538-ecddce2e614f':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=63','archive': 'https://ecdysis.org/content/dwca/NAU-NAUFEC_DwC-A.zip'}
        if uuid == '4aecf17c-154a-42fb-a3c0-f5e621c791e6':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=118','archive': 'https://ecdysis.org/content/dwca/LCDI-LERC_DwC-A.zip'}
        if uuid == '5e893602-84ca-4c8c-bac1-99111c777582':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=80','archive': 'https://ecdysis.org/content/dwca/TTU-TTU-Z_DwC-A.zip'}
        if uuid == '3c919328-94fd-4657-b81d-21f4707253ed':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=126','archive': 'https://ecdysis.org/content/dwca/UCD-BMEC_DwC-A.zip'}
        if uuid == 'e48bb88f-9594-461e-8230-522a3a5572fe':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=95','archive': 'https://ecdysis.org/content/dwca/SDNHM-SDMC_DwC-A.zip'}
        if uuid == '00df4fe2-0025-45ec-814a-36777155e077':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=119','archive': 'https://ecdysis.org/content/dwca/SDSU-TAC_DwC-A.zip'}
        if uuid == '87017793-00dc-4f5d-b95b-09e7d17327cc':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=128','archive': 'https://ecdysis.org/content/dwca/MSU-MSUC_DwC-A.zip'}
        if uuid == '7ed2ce52-26e5-4847-9563-955ae3e97455':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=106','archive': 'https://ecdysis.org/content/dwca/DCH-DCEC_DwC-A.zip'}
        if uuid == 'abb0a03c-4dcb-4f6f-a31d-55268f63d44c':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=123','archive': 'https://ecdysis.org/content/dwca/USNM-USNMENT_DwC-A.zip'}
        if uuid == '879d475f-4b76-4d18-8cf6-a7e5a6d44926':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=100','archive': 'https://ecdysis.org/content/dwca/GMNH-UGCA_DwC-A.zip'}
        if uuid == 'cf42855e-a54a-4488-a79e-beac086ba1d4':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=107','archive': 'https://ecdysis.org/content/dwca/UHIM-UHIM_DwC-A.zip'}
        if uuid == '96588aed-3b7a-4179-b92b-2159427f4fcb':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=110','archive': 'https://ecdysis.org/content/dwca/UVM-VTZT_DwC-A.zip'}
        if uuid == '4f436daa-01d5-4be6-b5c3-fdd255677536':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=98','archive': 'https://ecdysis.org/content/dwca/VPI-VTEC_DwC-A.zip'}
        if uuid == '34cad268-8226-4280-b637-dde38c82a29e':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=112','archive': 'https://ecdysis.org/content/dwca/WWU-WWUC_DwC-A.zip'}
        if uuid == 'b985f284-eac5-4efe-8a7c-c726cdf7cf33':
            return {'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=150','archive': 'https://ecdysis.org/content/dwca/UI-WFBM_DwC-A.zip'}
        return None
           

In [44]:
updatedRecord = lookupURL('abb0a03c-4dcb-4f6f-a31d-55268f63d44c')

updatedRecord

{'archive': 'https://ecdysis.org/content/dwca/USNM-USNMENT_DwC-A.zip',
 'eml': 'https://ecdysis.org/collections/datasets/emlhandler.php?collid=123'}

In [78]:

#Update Object

for entry in entry_ids:
    updatedRecord = None
    for item in current_rs[entry]:
        #print(item)
        for k, v in item.items():
            if k == 'recordids':
                print(v[0])
                updatedRecord = lookupURL(v[0])
                if updatedRecord is not None:
                    item['eml_link'] = updatedRecord['eml']
                    item['file_link'] = updatedRecord['archive']
                    print("updated object...")


            

992ca726-fd6e-4398-a8c6-07dcf1f71083
b235bd87-fd7a-4c52-a5f7-79008c393131
e3570181-82db-489c-b146-d8cb4b33e472
41b2f1af-9b4e-450d-8f6c-e5df18e49711
335643f7-6835-41fd-9dca-c6b8e7918780
741d3036-156f-49fa-b672-1d7aad6d2810
c7263105-7706-4d44-9384-8d0ef527a33a
1ddcac88-7c7d-44c3-8bc9-3435aceeb615
b8c8c26f-e836-4227-88bb-fa6b2446edb3
3ae42fd2-0827-4f35-9ec5-89d26914b5a5
2bd20e20-fae7-46e6-a496-5a9d71e962a2
16fca9c2-f622-4cb1-aef0-3635a7be5aeb
cda97de1-b5cc-4963-b497-a1ee6312cf77
37bed406-b435-4539-b319-be2fdfb75b32
0ca2afb9-24f1-48da-b5aa-e3b1727236f5
ed38f34c-6591-4043-8e67-1f4597e2b903
a9059e97-59da-48f3-8f7f-aec3ddfac069
bf3e5fc5-8a24-4a59-88e6-cb12cae48258
25b650b5-ada6-47ae-b438-b33707f97724
7571ba64-4f12-4af9-a9ab-d2eba06b3af6
de06a5b7-ffe2-40c4-b376-bb9b779bbdc9
85d00be2-3f99-49c2-9185-3942fca4c5dc
847b0b2e-b6d5-4373-8c09-0ab3bc9b8376
324e42d3-bb41-4cff-b067-7bd55983d5a7
ff59ebc1-27ef-4137-adec-ca4b673496c7
17fb5c4e-64ed-4da9-84ec-af6ae89b47c9
cf11b630-dbf1-4ef3-b7bd-67e11c14f60d
2